In [ ]:
def strong_odd_chromatic_number_LP(graph, max_colors=None):
    from sage.numerical.mip import MixedIntegerLinearProgram

    n = graph.num_verts()
    vertices = list(graph.vertices())
    edges = list(graph.edges(labels=False))

    if max_colors is None:
        max_colors = n

    mip = MixedIntegerLinearProgram(maximization=False)
    x = mip.new_variable(binary=True)
    y = mip.new_variable(binary=True)
    z = mip.new_variable(integer=True, nonnegative=True)
    t = mip.new_variable(real=True, nonnegative=True)

    # Ciljna funkcija
    mip.set_objective(t[0])

    # Pogoj 1: Vsako vozlišče ima eno barvo
    for u in vertices:
        mip.add_constraint(sum(x[u, i] for i in range(1, max_colors + 1)) == 1)

    # Pogoj 2: Sosednja vozlišča imajo različne barve
    for u, v in edges:
        for i in range(1, max_colors + 1):
            mip.add_constraint(x[u, i] + x[v, i] <= 1)

    # Pogoj 3: Štetje barv
    for u in vertices:
        for i in range(1, max_colors + 1):
            mip.add_constraint(i * x[u, i] <= t[0])

    # Pogoj 4: Katere barve se pojavijo med sosedi
    for u in vertices:
        for i in range(1, max_colors + 1):
            mip.add_constraint(sum(x[v, i] for v in graph.neighbors(u)) <= n * y[u, i])

    # Pogoj 5: Barve se morajo pojaviti liho mnogokrat
    for u in vertices:
        for i in range(1, max_colors + 1):
            mip.add_constraint(
                sum(x[v, i] for v in graph.neighbors(u)) == y[u, i] + 2 * z[u, i]
            )

    # Reševanje problema
    mip.solve()

    # Pridobitev optimalne vrednosti
    chromatic_number = int(mip.get_objective_value())

    return chromatic_number
